### 读取nc文件

In [17]:
pip install parmed

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 7.9 MB/s eta 0:00:0000:0100:010m
  Preparing metadata (setup.py) ... done
  Created wheel for parmed: filename=ParmEd-4.1.0-cp39-cp39-linux_x86_64.whl size=1250041 sha256=fc70aed221839313ea1140dcce21166e14918ff1a4d6cd631f707c31f41cebd0
  Stored in directory: /home/ntu/.cache/pip/wheels/2a/eb/6a/8ec682dfe161fa89e821756f0c7244f820be751556a10c3b38
Successfully built parmed
Note: you may need to restart the kernel to use updated packages.


In [23]:
import netCDF4
from netCDF4 import Dataset
import parmed as pmd

In [31]:
data=Dataset('../data/drug_1_no_solvent.nc')
data


AttributeError: NetCDF: Attribute not found

In [27]:
#查看nc文件中的变量
print(data.variables.keys())
print('---------------------------------------')

#查看每个变量的信息
print(data.variables['coordinates'])
print('---------------------------------------')

dict_keys(['time', 'spatial', 'coordinates', 'cell_spatial', 'cell_angular', 'cell_lengths', 'cell_angles'])
---------------------------------------
<class 'netCDF4._netCDF4.Variable'>
float32 coordinates(frame, atom, spatial)
    units: angstrom
unlimited dimensions: frame
current shape = (5000, 20, 3)
filling on, default _FillValue of 9.969209968386869e+36 used
---------------------------------------


In [28]:
structure = pmd.load_file(data)

# # 获取原子标识符
# for atom in structure.atoms:
#     atom_id = atom.idx + 1  # 原子索引从0开始，所以加1以使编号从1开始
#     print("Atom", atom_id, ":", atom.name)



AttributeError: NetCDF: Attribute not found

### 查看蛋白的原子种类

In [10]:
! conda install -c conda-forge mdanalysis --y

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.3
  latest version: 4.12.0

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - mdanalysis


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    biopython-1.79             |   py37h5e8e339_1         2.6 MB  conda-forge
    ca-certificates-2021.10.8  |       ha878542_0         139 KB  conda-forge
    certifi-2021.10.8          |   py37h89c1867_1         145 KB  conda-forge
    cftime-1.6.0               |   py37h6c7ee08_0         211 KB  conda-forge
    curl-7.78.0                |       hea6ffbf_0    

In [11]:
import MDAnalysis as mda

In [12]:
u = mda.Universe('../data/com_wat.pdb')
len(u.atoms)

/opt/conda/lib/python3.7/site-packages/MDAnalysis/topology/PDBParser.py:317: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "


32323

In [56]:
# 去掉水和钠离子，德睿数据中没有保留金属离子，最后选取ligand
u = u.select_atoms("not resname WAT and not resname Na+")
u_ligand = u.select_atoms("resname LIA")
u_protein = u.select_atoms("not resname LIA")
print(len(u), len(u_protein), len(u_ligand))

5317 5284 33


In [62]:
import torch
dist = torch.cdist(torch.tensor(u_protein.atoms.positions), torch.tensor(u_ligand.atoms.positions))
ind = (torch.sum(dist < 10, dim=-1) > 0)
ind = torch.cat((ind, torch.ones(len(u_ligand)))).bool().numpy()
sum(ind)

894

In [11]:
with mda.Writer("../data/1TBF.pdb", len(u)) as W:
    W.write(u)

### 和标准PDB数据集比对

In [11]:
pdb = mda.Universe('../data/1TBF.pdb')
len(pdb.atoms) 

2698

### 写入xtc文件

In [15]:
u.atoms.positions.shape

(5317, 3)

In [13]:
u.atoms.names

array(['N', 'H1', 'H2', ..., 'C32', 'C33', 'C34'], dtype=object)

In [14]:
nc_obj.variables['coordinates'][0].shape

(32323, 3)

In [16]:
with mda.Writer("1TBF.xtc", len(u)) as W:
    for ts in nc_obj.variables['coordinates'][:, :len(u)]:
        u.atoms.positions = ts
        W.write(u)

/opt/conda/lib/python3.7/site-packages/MDAnalysis/coordinates/base.py:892: UserWarning: Reader has no dt information, set to 1.0 ps
  warnings.warn("Reader has no dt information, set to 1.0 ps")


In [40]:
tra = mda.Universe('1TBF.xtc')
tra.trajectory

### 读取MioIO上的文件

In [39]:
! pip install boto3

     |████████████████████████████████| 132 kB 382 kB/s eta 0:00:01
  Using cached jmespath-1.0.0-py3-none-any.whl (23 kB)
  Using cached s3transfer-0.5.2-py3-none-any.whl (79 kB)
     |████████████████████████████████| 8.6 MB 1.3 MB/s eta 0:00:01


In [63]:
import os
import torch
import numpy as np 
import boto3
from botocore.client import Config

In [64]:
BUCKET_NAME="md-data"
FOLDER_NAME="junjie/md/build"
s3 = boto3.client('s3', endpoint_url='http://minio-headless.minio.svc.cluster.local:9000',
                  aws_access_key_id="W6P4O6BRH8TD74ZLSALW",
                  aws_secret_access_key="WvEegImgTibaBB+5IZJ7HCNK7GXvNV0sHcEnJy07",
                  config=Config(signature_version='s3v4'))

In [65]:
s3.list_buckets()['Buckets']

[{'Name': 'md-data',
  'CreationDate': datetime.datetime(2022, 2, 11, 3, 4, 11, 175000, tzinfo=tzlocal())},
 {'Name': 'mindrank-public',
  'CreationDate': datetime.datetime(2022, 1, 20, 13, 46, 45, 7000, tzinfo=tzlocal())}]

In [66]:
result = s3.list_objects(Bucket=BUCKET_NAME, Prefix="%s/" % FOLDER_NAME, Delimiter='/')
name = []
for o in result.get('CommonPrefixes'):
    name.append(o.get('Prefix').split('/')[-2])
print('PDB name:', name) 

PDB name: ['1TBF', '1TXI', '1ZKL', '1ZP5', '2E1Q', '2GH5', '2I0E', '2JED', '2JIF', '2NO6', '2Z5X', '3B6H', '3D4S', '3DPK', '3FVO', '3I8V', '3IAR', '3IW4', '3JZB', '3LW0', '3OLL', '3QXM', '3ROD', '3TKM', '3W2T', '4DJH', '4IAQ', '4IB4', '4MUW', '4NQD', '4PXZ', '4QTB', '4RWD', '4UDA', '4UXQ', '5AFJ', '5AX3', '5C37', '5CGD', '5DIQ', '5DSG', '5DYY', '5EDU', '6R4V', '6SSQ', '6WV3', '6X40', '7AOS', '7AYM', '7B0V', '7BR3', '7BVQ', '7BW1', '7C7S', '7CMV', '7CX3', '7DFW', '7DHL', '7EO4', '7JVP', '7JVR', '7LRC', '7VNR']


In [ ]:
# 遍历所有的pdb
for i in name:
    objects_in_bucket = s3.list_objects_v2(Bucket=BUCKET_NAME, Prefix="%s/" % FOLDER_NAME + i)['Contents']
    for obj in objects_in_bucket:
        key = obj['Key']
        if key.endswith('prod1.nc') or key.endswith('com_wat.pdb'):
            print("Retrieving file %s" % key)
        else:
            continue        
        try:
            os.mkdir(os.path.join('md_data/', i))
        except:
            pass
        
        if key.endswith('prod1.nc'):
            s3.download_file(BUCKET_NAME, '/' + key, 'md_data/' + i + '/' + i + '.nc')
        elif key.endswith('com_wat.pdb'):
            s3.download_file(BUCKET_NAME, '/' + key, 'md_data/' + i + '/' + i + '.pdb') 

In [ ]:
atom_dict = {'H':1, 'C':6, 'N':7, 'O':8}
file_path = 'md_data/'
file_list = os.listdir(file_path)

for file in file_list:
    if len(file) > 4:
        continue
    nc_obj = Dataset(file_path + file + '/' + file + '.nc')
    u = mda.Universe(file_path + file + '/' + file + '.pdb')
    u = u.select_atoms("not resname WAT and not resname Na+")
    
    u_ligand = u.select_atoms("resname LIA")
    u_protein = u.select_atoms("not resname LIA") 
    
    # atom3d选的是6A距离
    dist = torch.cdist(torch.tensor(u_protein.atoms.positions), torch.tensor(u_ligand.atoms.positions))
    mask = (torch.sum(dist < 6, dim=-1) > 0)
    mask = torch.cat((mask, torch.ones(len(u_ligand)))).bool().numpy()
    
    atoms = [x[0] for x in u.atoms.names[:len(u)]]
    tr = {'name': file}
    tr['z'] = atoms
    tr['R'] = nc_obj.variables['coordinates'][:, :len(u)][:, mask]    
    # 单个保存，避免内存爆炸
    with open(f'data/{file}.npy', 'wb') as f:
        np.save(f, tr)
    
print(f'{len(file_list)} proteins are saved.')

/opt/conda/lib/python3.7/site-packages/MDAnalysis/topology/PDBParser.py:317: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "
/opt/conda/lib/python3.7/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: AL
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


In [118]:
a = np.load('md_pdb.npy', allow_pickle=True)